<a href="https://colab.research.google.com/github/deborahchang/projects/blob/Campaign-Financing-on-Election-Outcomes/Random_Forest_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score

In [ ]:
# Load Dem Candidates 2016
drive.mount('/content/drive')
dem_candidates_path = '/content/drive/MyDrive/144Project/dem_candidates.csv'
dem_candidates = pd.read_csv(dem_candidates_path)
print(dem_candidates.columns)
dem_candidates.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Index(['Candidate', 'State', 'District', 'Office Type', 'Race Type',
       'Race Primary Election Date', 'Primary Status', 'Primary Runoff Status',
       'General Status', 'Partisan Lean', 'Primary %', 'Won Primary', 'Race',
       'Veteran?', 'LGBTQ?', 'Elected Official?', 'Self-Funder?', 'STEM?',
       'Obama Alum?', 'Party Support?', 'Emily Endorsed?',
       'Guns Sense Candidate?', 'Biden Endorsed?', 'Warren Endorsed? ',
       'Sanders Endorsed?', 'Our Revolution Endorsed?',
       'Justice Dems Endorsed?', 'PCCC Endorsed?', 'Indivisible Endorsed?',
       'WFP Endorsed?', 'VoteVets Endorsed?', 'No Labels Support?'],
      dtype='object')


,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Partisan Lean,...,Biden Endorsed?,Warren Endorsed?,Sanders Endorsed?,Our Revolution Endorsed?,Justice Dems Endorsed?,PCCC Endorsed?,Indivisible Endorsed?,WFP Endorsed?,VoteVets Endorsed?,No Labels Support?
0,Anthony White (Alabama),AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Christopher Countryman,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Doug ""New Blue"" Smith",AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,James C. Fields,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sue Bell Cobb,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
lobby_path = '/content/drive/MyDrive/144Project/lobbyist_bundle.csv'
lobby = pd.read_csv(lobby_path)
print(lobby.columns)
lobby = lobby.rename(columns={'Committee_Election_State':'State'}).dropna()
lobby['Total_Contribution'] = lobby['Quarterly_Contribution'] + lobby['Semi_Annual_Contribution']
lobby = lobby[['State', 'Total_Contribution']]
lobby = lobby.groupby('State')['Total_Contribution'].sum().reset_index()
lobby

Index(['Committee_Id', 'Committee_Name', 'Link_Image',
       'Committee_Election_State', 'Committee_Election_District',
       'Report_Type', 'Receipt_Date', 'Coverage_Start_Date',
       'Coverage_End_Date', 'Quarterly_Contribution',
       'Semi_Annual_Contribution'],
      dtype='object')


,State,Total_Contribution
0,AL,84450.00
1,AZ,1259414.00
2,CA,1885454.20
3,CO,2100231.28
4,DC,23800.00
5,DE,101200.00
6,GA,418812.53
7,HI,538462.00
8,IL,967320.50
9,KS,49078.68


In [ ]:
dem_candidates = pd.merge(dem_candidates, lobby, on='State', how='left')
dem_candidates.head()

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Partisan Lean,...,Warren Endorsed?,Sanders Endorsed?,Our Revolution Endorsed?,Justice Dems Endorsed?,PCCC Endorsed?,Indivisible Endorsed?,WFP Endorsed?,VoteVets Endorsed?,No Labels Support?,Total_Contribution
0,Anthony White (Alabama),AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84450.0
1,Christopher Countryman,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84450.0
2,"Doug ""New Blue"" Smith",AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84450.0
3,James C. Fields,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84450.0
4,Sue Bell Cobb,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,None,None,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84450.0


In [ ]:
selected_features = [
    'Total_Contribution',
    'Partisan Lean',
    'Self-Funder?',
    'Obama Alum?',
    'Party Support?',
    'Emily Endorsed?',
    'Guns Sense Candidate?',
    'Biden Endorsed?',
    'Warren Endorsed? ',
    'Sanders Endorsed?',
    'Our Revolution Endorsed?',
    'Justice Dems Endorsed?',
    'PCCC Endorsed?',
    'Indivisible Endorsed?',
    'WFP Endorsed?',
    'VoteVets Endorsed?',
    'No Labels Support?',
    'Won Primary'  # Target variable
]

dem_candidates = dem_candidates[selected_features]
print(dem_candidates.isnull().sum())


KeyError: ignored

In [ ]:
mean_contributions = dem_candidates['Total_Contribution'].mean()
dem_candidates['Total_Contribution'].fillna(mean_contributions, inplace=True)

dem_candidates = dem_candidates.fillna(0)

yes_no_mapping = {'Yes': 1, 'No': 0}
dem_candidates[selected_features] = dem_candidates[selected_features].applymap(lambda x: yes_no_mapping.get(x, x))
dem_candidates.head(20)




,Total_Contribution,Partisan Lean,Self-Funder?,Obama Alum?,Party Support?,Emily Endorsed?,Guns Sense Candidate?,Biden Endorsed?,Warren Endorsed?,Sanders Endorsed?,Our Revolution Endorsed?,Justice Dems Endorsed?,PCCC Endorsed?,Indivisible Endorsed?,WFP Endorsed?,VoteVets Endorsed?,No Labels Support?,Won Primary
0,8.445000e+04,-28.879999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8.445000e+04,-28.879999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,8.445000e+04,-28.879999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,8.445000e+04,-28.879999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8.445000e+04,-28.879999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,8.445000e+04,-28.879999,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
6,8.445000e+04,-30.680000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,8.445000e+04,-30.680000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,8.445000e+04,-33.080002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,8.445000e+04,-33.080002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
X = dem_candidates.drop(['Won Primary'], axis=1)
y = dem_candidates['Won Primary']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}

clf = RandomForestClassifier()

# Create the GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Retrieve the best model and hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Best Hyperparameters: {best_params}')
print(f'Accuracy: {accuracy}')

Best Hyperparameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 100}
Accuracy: 0.7361963190184049


In [ ]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X, y)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Display the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')


Accuracy: 0.9079754601226994
Confusion Matrix:
[[110   1]
 [ 14  38]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.99      0.94       111
           1       0.97      0.73      0.84        52

    accuracy                           0.91       163
   macro avg       0.93      0.86      0.89       163
weighted avg       0.91      0.91      0.90       163



In [ ]:
# Load Rep Candidates 2016
rep_candidates_path = '/content/drive/MyDrive/144Project/rep_candidates.csv'
rep_candidates = pd.read_csv(rep_candidates_path, encoding='latin-1')
rep_candidates.head()

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Primary %,...,NRA Endorsed?,Right to Life Endorsed?,Susan B. Anthony Endorsed?,Club for Growth Endorsed?,Koch Support?,House Freedom Support?,Tea Party Endorsed?,Main Street Endorsed?,Chamber Endorsed?,No Labels Support?
0,Mike Dunleavy,AK,Governor of Alaska,Governor,Regular,8/21/18,Advanced,None,On the Ballot,61.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Michael Sheldon,AK,Governor of Alaska,Governor,Regular,8/21/18,Lost,None,None,2.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mead Treadwell,AK,Governor of Alaska,Governor,Regular,8/21/18,Lost,None,None,31.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Darin Colbry,AK,Governor of Alaska,Governor,Regular,8/21/18,Lost,None,None,0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Thomas Gordon,AK,Governor of Alaska,Governor,Regular,8/21/18,Lost,None,None,1.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rep_candidates = pd.merge(rep_candidates, lobby, on='State', how='left')

print(rep_candidates.columns)
rep_candidates.head()

Index(['Candidate', 'State', 'District', 'Office Type', 'Race Type',
       'Race Primary Election Date', 'Primary Status', 'Primary Runoff Status',
       'General Status', 'Primary %', 'Won Primary', 'Rep Party Support?',
       'Trump Endorsed?', 'Bannon Endorsed?', 'Great America Endorsed?',
       'NRA Endorsed?', 'Right to Life Endorsed?',
       'Susan B. Anthony Endorsed?', 'Club for Growth Endorsed?',
       'Koch Support?', 'House Freedom Support?', 'Tea Party Endorsed?',
       'Main Street Endorsed?', 'Chamber Endorsed?', 'No Labels Support?',
       'Total_Contribution'],
      dtype='object')


,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Primary %,...,Right to Life Endorsed?,Susan B. Anthony Endorsed?,Club for Growth Endorsed?,Koch Support?,House Freedom Support?,Tea Party Endorsed?,Main Street Endorsed?,Chamber Endorsed?,No Labels Support?,Total_Contribution
0,Mike Dunleavy,AK,Governor of Alaska,Governor,Regular,8/21/18,Advanced,None,On the Ballot,61.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Michael Sheldon,AK,Governor of Alaska,Governor,Regular,8/21/18,Lost,None,None,2.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mead Treadwell,AK,Governor of Alaska,Governor,Regular,8/21/18,Lost,None,None,31.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Darin Colbry,AK,Governor of Alaska,Governor,Regular,8/21/18,Lost,None,None,0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Thomas Gordon,AK,Governor of Alaska,Governor,Regular,8/21/18,Lost,None,None,1.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rep_features = [
    'Total_Contribution',
    'Rep Party Support?',
    'Trump Endorsed?',
    'Bannon Endorsed?',
    'Great America Endorsed?',
    'NRA Endorsed?',
    'Right to Life Endorsed?',
    'Susan B. Anthony Endorsed?',
    'Club for Growth Endorsed?',
    'Koch Support?',
    'House Freedom Support?',
    'Tea Party Endorsed?',
    'Main Street Endorsed?',
    'Chamber Endorsed?',
    'No Labels Support?',
    'Won Primary'  # Target variable
]

rep_candidates = rep_candidates[rep_features]
print(rep_candidates.isnull().sum())


Total_Contribution            126
Rep Party Support?            636
Trump Endorsed?               703
Bannon Endorsed?              731
Great America Endorsed?       738
NRA Endorsed?                 705
Right to Life Endorsed?       473
Susan B. Anthony Endorsed?    644
Club for Growth Endorsed?     649
Koch Support?                 663
House Freedom Support?        681
Tea Party Endorsed?           674
Main Street Endorsed?         676
Chamber Endorsed?             747
No Labels Support?            750
Won Primary                     0
dtype: int64


In [ ]:
rep_candidates['Total_Contribution'].fillna(mean_contributions, inplace=True)

rep_candidates = rep_candidates.fillna(0)

yes_no_mapping = {'Yes': 1, 'No': 0}
rep_candidates[rep_features] = rep_candidates[rep_features].applymap(lambda x: yes_no_mapping.get(x, x))
rep_candidates.head(20)



<ipython-input-40-14b5cd741c29>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rep_candidates['Total_Contribution'].fillna(mean_contributions, inplace=True)


,Total_Contribution,Rep Party Support?,Trump Endorsed?,Bannon Endorsed?,Great America Endorsed?,NRA Endorsed?,Right to Life Endorsed?,Susan B. Anthony Endorsed?,Club for Growth Endorsed?,Koch Support?,House Freedom Support?,Tea Party Endorsed?,Main Street Endorsed?,Chamber Endorsed?,No Labels Support?,Won Primary
0,1.668323e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1.668323e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.668323e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.668323e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.668323e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1.668323e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1.668323e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1.259414e+06,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0
8,1.259414e+06,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9,1.259414e+06,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_rep = rep_candidates.drop(['Won Primary'], axis=1)
y_rep = rep_candidates['Won Primary']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_rep, y_rep, test_size=0.2, random_state=42)

In [ ]:
# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}

clf = RandomForestClassifier()

# Create the GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Retrieve the best model and hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Best Hyperparameters: {best_params}')
print(f'Accuracy: {accuracy}')

Best Hyperparameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Accuracy: 0.7096774193548387


In [ ]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')

Accuracy: 0.7096774193548387
Confusion Matrix:
[[97 14]
 [31 13]]
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.87      0.81       111
           1       0.48      0.30      0.37        44

    accuracy                           0.71       155
   macro avg       0.62      0.58      0.59       155
weighted avg       0.68      0.71      0.69       155

